In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [10]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(
    temperature=0.1
)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator='\n',
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader("./files/chapter_one.docx")
docs = loader.load_and_split(text_splitter=splitter)
embeddings = OpenAIEmbeddings()

# cache_dir에 있는 embed를 확인하고 업으면 OpenAIEmbeddings 사용
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir
)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriever = vectorstore.as_retriever()

# 요구사항
# list of docs -> Victory Mansions을 묘사하는 것과 관련된 document list를 retriever가 반환
# for doc in list of docs | promt | llm -> 모든 document에 대한 prompt를 만들어서 llm에 전달. 질문: "이 doc을 읽고 question에 답하는 데에 관련이 있는 중요한 정보를 추출해줘"
# for response in list of llms response | put them all together -> response를 합해서 하나의 긴 document(final doc)
# final doc | prompt | llm -> final doc이 prompt에 입력되어 전달 + llm이 질문에 대답


# chain.invoke("Describe Victory Mansions.")